In [1]:
import cv2 as cv
import numpy as np
import os
import pandas as pd

In [2]:
def framing(video_path):

    cap = cv.VideoCapture(video_path)
    fps = int(cap.get(5))
    print("fps:", fps)
    if(fps==0):
        print("Not available")
        return 0

    res_fr = []
    while True:
        ret, frame = cap.read()
        if ret:
            resized_frame = cv.resize(frame, (224, 224), interpolation= cv.INTER_LINEAR)
            res_fr.append(resized_frame)
        else:
            break
            
    return res_fr

In [3]:
def keyframeDetection(video_path):
    resized_frames  = framing(video_path)
    
    if(len(resized_frames)==0):
        print("Cannot compare video because of zero fps.")
        return
    
    else:
        video_name = (video_path.split('/')[-1]).split('.')[0]
        keyframePath = 'keyframes/'+video_name
        if not os.path.exists(keyframePath):
            os.makedirs(keyframePath)
        
        diff = []
        for i in range(0,len(resized_frames)-1):
            diff.append(cv.absdiff(resized_frames[i], resized_frames[i+1]))

        mn_all_frames = np.mean(diff)         #This gives mean
        st_d_all_frames = np.std(diff)        #This gives standard deviation
        a = 8                                 #Setting a random value we can modify it to any value 
        ts = mn_all_frames + (a * st_d_all_frames)   # global threshold
        
        local_ts = []
        for i in range(len(diff)):
            mn = np.mean(diff[i])             #Calculating the mean for each frame
            st_d = np.std(diff[i])            #Calculating the standard deviation for each frame
            fr_ts = mn + (5*st_d)             #Finding the threshold values for each frame
            local_ts.append([i,fr_ts])
            

        frame_num = []
        count = 0
        for i,l_tr in(local_ts):
            if l_tr >= ts:                   #Comparing the threshold values to the standard threshold/global threshold values
                cv.imwrite(keyframePath+'/'+str(i)+'.jpg',resized_frames[i]) 
                frame_num.append(i)
                count+=1

        print("Total Number of keyframes saved: {}".format(count))    
        return frame_num, keyframePath

In [4]:
def imageSim(img1_path, img2_path):
    
    img1 = cv.imread(img1_path)
    img2 = cv.imread(img2_path)
    
    finder = cv.xfeatures2d.SIFT_create()
    kp_1, desc_1 = finder.detectAndCompute(img1, None)
    kp_2, desc_2 = finder.detectAndCompute(img2, None)
    
    # FLANN parameters
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)   # or pass empty dictionary

    flann = cv.FlannBasedMatcher(index_params,search_params)

    matches = flann.knnMatch(desc_1,desc_2,k=2)
    
    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append([m])
    a=len(good)
    percent=(a*100)/len(kp_2)

#     return kp_1, kp_2, desc_1, desc_2, matches
    return percent

In [5]:
k1, k2, d1, d2, m = imageSim("keyframes/test5/35.jpg", "keyframes/test5/37.jpg")
print(len(m))
m

221


[[<DMatch 00000236437C9B10>, <DMatch 00000236437C9890>],
 [<DMatch 00000236437C98B0>, <DMatch 00000236437C9BD0>],
 [<DMatch 00000236437C9BB0>, <DMatch 00000236437C9B50>],
 [<DMatch 00000236437C99F0>, <DMatch 00000236437C9AB0>],
 [<DMatch 00000236437C9390>, <DMatch 00000236437C9C10>],
 [<DMatch 00000236437C9C30>, <DMatch 00000236437C9BF0>],
 [<DMatch 00000236437C99B0>, <DMatch 00000236437C9C70>],
 [<DMatch 00000236437C9CB0>, <DMatch 00000236437C9CD0>],
 [<DMatch 00000236437C9D10>, <DMatch 00000236437C9CF0>],
 [<DMatch 00000236437C9D30>, <DMatch 00000236437C9A90>],
 [<DMatch 00000236437C9C50>, <DMatch 00000236437C9A70>],
 [<DMatch 00000236437C9A10>, <DMatch 00000236437C96F0>],
 [<DMatch 00000236437C98D0>, <DMatch 00000236437C9990>],
 [<DMatch 00000236437C9D70>, <DMatch 00000236437C9870>],
 [<DMatch 00000236437C9D90>, <DMatch 00000236437C9DB0>],
 [<DMatch 00000236437C9DD0>, <DMatch 00000236437C9DF0>],
 [<DMatch 00000236437C9E10>, <DMatch 00000236437C9E30>],
 [<DMatch 00000236437C9E50>, <D

In [21]:
pts = [k1[idx].pt for idx in range(len(k1))]
pts

[(23.62249183654785, 74.35792541503906),
 (25.15420150756836, 71.4920425415039),
 (32.12788772583008, 53.19933319091797),
 (32.12788772583008, 53.19933319091797),
 (32.71330642700195, 64.78168487548828),
 (32.73833465576172, 118.91586303710938),
 (32.912086486816406, 169.42152404785156),
 (33.12593460083008, 119.23184204101562),
 (34.269344329833984, 95.5491714477539),
 (34.269344329833984, 95.5491714477539),
 (35.19859313964844, 115.65826416015625),
 (35.20262908935547, 81.25641632080078),
 (35.2155876159668, 132.3334197998047),
 (35.66029357910156, 138.7112579345703),
 (36.80313491821289, 168.89109802246094),
 (36.894779205322266, 85.0401382446289),
 (36.894779205322266, 85.0401382446289),
 (36.94076156616211, 166.69683837890625),
 (37.024208068847656, 172.93492126464844),
 (37.11299514770508, 92.57828521728516),
 (37.180938720703125, 77.82891082763672),
 (37.180938720703125, 77.82891082763672),
 (37.32421875, 103.19446563720703),
 (37.47154235839844, 96.28612518310547),
 (37.7953338

In [31]:
d1

array([[  0.,   0.,   0., ...,   1.,  78.,  65.],
       [  0.,   0.,   0., ...,   6.,  15.,  26.],
       [  0.,   0.,   0., ...,   1.,   9.,   3.],
       ...,
       [ 21.,  10.,   3., ...,   0.,   0.,   1.],
       [150.,  27.,   0., ...,   4.,  19.,   7.],
       [  5.,   0.,   0., ...,   0.,   0.,   0.]], dtype=float32)

In [43]:
print(len(d1[0]))
print(d1[0][8])
print(d1[0])

128
19.0
[  0.   0.   0.   0.   0.   0.   0.   0.  19.   0.   0.   0.   0.   0.
   4.  29.  23.   0.   0.   0.   0.   1. 141. 104.   0.   0.   0.   0.
   0.   5. 141.  10.   1.   0.   0.   0.   0.   0.   0.   0. 135.   8.
   0.   0.   0.   0.   1.  23. 141.  72.  14.   0.   0.   0.  72. 109.
  17.  47.  35.   0.   0.   1. 141.  27.   1.   0.   0.   0.   0.   0.
   0.   0. 141.   3.   0.   0.   0.   0.   0.  17. 141.  28.   5.   0.
   0.   0.   2.  58.  69.  26.  13.   0.   0.   0.   2.  55.   2.   0.
   0.   0.   0.   0.   0.   0. 141.   0.   0.   0.   0.   0.   0.  34.
 141.   3.   0.   0.   0.   1.  40. 126.  51.   8.   1.   0.   0.   1.
  78.  65.]


In [44]:
len(d1)

221

In [45]:
len(d2)

249

In [5]:
def videoSimilarity(video1_path, video2_path):
    v1_keyframe_num, keyframe1_path = keyframeDetection(video1_path)
    v2_keyframe_num, keyframe2_path = keyframeDetection(video2_path)
    
    sim_mat = []
    count=0
    for i in v1_keyframe_num:
        row = []
        p1 = keyframe1_path+'/'+str(i)+'.jpg'
        for j in v2_keyframe_num:  
            p2 = keyframe2_path+'/'+str(j)+'.jpg'
            sim = imageSim(p1, p2)
            row.append(sim)

        count = count+1
        print("row",count, end='\r')
        sim_mat.append(row)
        
    sim_df = pd.DataFrame(sim_mat)
#     sim_percent = ((sim_df.max(axis=1).sum())/len(sim_df))
    return sim_df

### BlackBorder & Speeder Videos

In [6]:
df = videoSimilarity("test-videos/test5.mp4", "test-videos/test6.mp4")
df

fps: 30
Total Number of keyframes saved: 31
fps: 30
Total Number of keyframes saved: 14


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,10.062893,9.230769,15.546218,8.550186,14.529915,6.644518,2.173913,3.267974,1.408451,3.571429,2.985075,3.252033,5.851064,4.964539
1,16.666667,11.153846,11.764706,9.293680,11.111111,5.315615,3.623188,2.941176,0.938967,2.380952,1.791045,6.504065,2.659574,4.255319
2,19.182390,10.769231,13.025210,8.550186,8.119658,4.983389,3.260870,2.941176,0.938967,1.785714,2.089552,4.065041,2.127660,7.801418
3,36.792453,14.615385,17.226891,10.408922,9.829060,7.308970,3.623188,3.594771,1.408451,4.761905,3.880597,8.130081,3.191489,6.382979
4,20.125786,17.692308,9.663866,10.780669,10.256410,7.308970,2.898551,4.901961,1.408451,3.571429,2.686567,4.878049,1.595745,7.092199
5,12.264151,33.846154,17.226891,8.550186,10.256410,6.312292,2.173913,3.594771,2.347418,2.380952,2.388060,3.252033,1.063830,4.255319
6,9.119497,15.384615,42.436975,9.293680,11.965812,4.318937,4.710145,3.267974,1.408451,1.190476,2.089552,4.878049,5.319149,2.836879
7,6.918239,7.307692,12.184874,32.342007,19.658120,7.973422,4.710145,4.248366,3.286385,3.571429,2.388060,1.626016,6.382979,6.382979
8,5.974843,5.384615,13.865546,15.613383,27.777778,11.960133,7.246377,4.575163,2.816901,2.380952,1.791045,2.439024,5.319149,6.382979
9,2.830189,5.384615,11.344538,11.152416,19.658120,30.564784,11.594203,7.516340,1.408451,2.380952,3.283582,4.878049,2.659574,3.546099


In [25]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
max_df.sum()/len(max_df)

38.307631032203716

In [8]:
print(df.max(axis=1).sum()/len(df))

26.036791077561567


### Videos with Jumbled Frames

In [26]:
df = videoSimilarity("test-videos/test7.mp4", "test-videos/test8.mp4")
df

fps: 30
Total Number of keyframes saved: 70
fps: 25
Total Number of keyframes saved: 56


,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,11.111111,1.351351,1.960784,2.542373,2.702703,1.265823,1.990050,1.626016,1.680672,0.858369,...,7.808564,7.073171,4.952830,8.040201,8.000000,6.648199,4.722222,3.846154,3.278689,1.990050
1,14.285714,1.013514,3.137255,3.389831,2.702703,2.109705,3.980100,1.626016,2.521008,3.433476,...,9.571788,7.804878,8.018868,9.296482,6.588235,8.310249,5.000000,5.029586,3.278689,1.492537
2,14.285714,1.689189,2.745098,2.118644,2.316602,3.375527,3.980100,2.032520,2.100840,0.858369,...,9.068010,6.829268,7.547170,9.296482,6.117647,7.479224,3.888889,5.325444,3.005464,3.482587
3,4.761905,2.364865,4.313725,2.966102,3.474903,2.531646,4.477612,1.219512,2.521008,2.575107,...,9.571788,9.268293,7.547170,10.804020,6.823529,7.479224,4.722222,4.142012,4.098361,1.990050
4,22.222222,1.351351,3.137255,4.237288,4.247104,2.953586,2.985075,2.032520,1.260504,1.287554,...,10.075567,7.804878,8.962264,9.296482,7.058824,9.695291,6.388889,2.958580,4.371585,2.985075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,9.523810,1.351351,0.392157,2.966102,3.474903,2.531646,3.482587,2.439024,1.680672,1.716738,...,3.778338,4.390244,4.481132,3.768844,3.294118,3.878116,3.888889,3.550296,1.366120,1.990050
66,7.936508,1.689189,1.176471,3.813559,2.702703,2.109705,2.985075,1.626016,2.100840,0.858369,...,6.045340,4.878049,4.952830,4.271357,3.058824,4.986150,4.722222,2.662722,2.732240,0.995025
67,4.761905,1.351351,2.352941,1.694915,3.088803,2.531646,2.487562,2.845528,2.521008,2.145923,...,7.052897,6.829268,4.952830,5.527638,5.882353,5.540166,5.000000,2.662722,3.278689,2.985075
68,12.698413,1.351351,2.352941,1.694915,1.930502,2.531646,1.990050,2.439024,1.680672,0.858369,...,7.556675,6.097561,4.952830,6.783920,7.294118,6.648199,4.722222,4.142012,4.371585,2.487562


In [27]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
max_df.sum()/len(max_df)

67.49689185600948

In [10]:
print(df.max(axis=1).sum()/len(df))

58.683826629416835


### -------------------------------------

In [28]:
df = videoSimilarity("test-videos/test9.mp4", "test-videos/test10.mp4")
df

fps: 30
Total Number of keyframes saved: 16
fps: 25
Total Number of keyframes saved: 16


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,77.910448,4.137931,17.897727,2.857143,2.597403,1.960784,2.194357,2.487562,1.030928,1.149425,1.632653,0.865801,0.925926,2.072539,1.020408,0.549451
1,3.880597,72.413793,4.261364,8.571429,0.000000,0.653595,0.626959,9.452736,0.000000,0.000000,0.816327,1.298701,1.388889,1.554404,0.680272,0.549451
2,16.716418,3.103448,75.284091,3.174603,2.922078,3.267974,0.000000,0.995025,1.030928,1.149425,1.632653,0.865801,2.314815,3.108808,0.340136,1.098901
3,2.388060,8.965517,3.125000,75.873016,0.974026,3.267974,0.626959,2.985075,1.546392,1.436782,3.673469,2.164502,0.925926,1.554404,1.020408,1.098901
4,1.492537,5.862069,0.852273,0.952381,0.974026,0.653595,0.940439,77.611940,1.030928,0.862069,1.224490,1.731602,2.777778,1.036269,0.340136,1.098901
5,2.686567,0.689655,1.420455,1.269841,0.324675,4.575163,4.702194,3.980100,74.742268,60.344828,50.612245,40.692641,25.925926,18.652850,4.761905,9.890110
6,2.089552,0.689655,1.420455,1.269841,1.623377,4.575163,3.761755,4.477612,57.989691,78.160920,60.816327,46.753247,29.629630,20.207254,4.081633,8.241758
7,1.791045,1.034483,1.420455,1.269841,0.974026,5.228758,4.075235,2.985075,35.051546,45.402299,80.000000,58.441558,34.259259,24.870466,4.421769,7.692308
8,1.791045,0.344828,0.568182,0.634921,1.298701,6.535948,3.448276,3.980100,25.515464,33.333333,55.102041,83.982684,43.981481,34.196891,6.122449,7.142857
9,1.194030,0.344828,1.704545,3.809524,4.870130,2.614379,1.567398,1.492537,13.917526,16.954023,30.612245,39.826840,75.925926,48.704663,7.482993,13.186813


In [29]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
max_df.sum()/len(max_df)

77.24731107998174

In [24]:
print(df.max(axis=1).sum()/len(df))

72.55671677484551


### -------------------------------------

In [30]:
df = videoSimilarity("test-videos/test11.mp4", "test-videos/test12.mp4")
df

fps: 25
Total Number of keyframes saved: 32
fps: 30
Total Number of keyframes saved: 45


,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,1.098901,2.463054,3.982301,3.108808,3.017241,1.492537,0.342466,0.724638,2.583026,1.063830,...,2.030457,1.707317,1.412429,1.423488,0.327869,1.282051,4.214559,0.917431,1.424501,0.772201
1,2.197802,0.492611,0.442478,4.145078,1.293103,0.746269,0.342466,0.362319,1.107011,0.000000,...,1.015228,0.975610,0.282486,0.711744,0.655738,0.256410,0.383142,0.688073,0.284900,0.000000
2,1.098901,2.955665,3.097345,1.554404,6.034483,1.119403,3.767123,2.173913,2.214022,0.709220,...,0.761421,0.731707,1.129944,2.135231,1.967213,1.282051,0.766284,0.688073,0.569801,0.772201
3,0.549451,1.477833,2.212389,1.554404,3.448276,1.492537,0.684932,1.811594,1.107011,1.063830,...,4.822335,6.097561,7.344633,3.558719,7.868852,8.974359,3.448276,5.963303,5.982906,0.386100
4,0.000000,0.492611,2.212389,1.554404,1.293103,1.119403,0.342466,0.000000,1.845018,0.709220,...,1.522843,0.487805,1.412429,3.558719,1.639344,1.538462,1.149425,1.376147,2.849003,0.000000
5,1.098901,3.940887,2.212389,2.072539,0.862069,1.492537,1.712329,1.086957,1.845018,1.418440,...,7.614213,5.609756,7.627119,4.626335,5.573770,4.871795,5.747126,3.669725,5.413105,0.386100
6,1.098901,1.970443,2.212389,1.554404,1.293103,1.492537,0.342466,1.449275,0.738007,1.418440,...,1.522843,1.219512,1.412429,3.558719,2.622951,2.564103,1.915709,0.229358,1.139601,0.386100
7,2.747253,1.477833,3.982301,2.072539,1.724138,0.746269,1.712329,1.449275,1.476015,1.418440,...,9.644670,9.024390,13.276836,13.167260,7.868852,6.153846,11.111111,6.192661,7.407407,1.544402
8,1.098901,1.970443,2.212389,2.590674,0.431034,0.746269,0.684932,1.086957,0.369004,0.354610,...,7.614213,7.317073,8.757062,11.032028,15.409836,5.897436,13.409962,6.422018,9.686610,0.772201
9,1.098901,1.970443,2.654867,2.590674,2.586207,1.119403,2.054795,0.724638,2.583026,2.127660,...,26.142132,18.048780,18.079096,9.964413,15.737705,18.205128,15.708812,13.532110,12.535613,0.772201


In [31]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
max_df.sum()/len(max_df)

71.78798016381957

In [22]:
print(df.max(axis=1).sum()/len(df))

68.54149114384057


### -------------------------------------

In [10]:
df = videoSimilarity("test-videos/new-test2-og.mp4", "test-videos/new-test2-dup.mp4")
df

fps: 30
Total Number of keyframes saved: 68
fps: 30
Total Number of keyframes saved: 52


,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,1.941748,2.380952,0.000000,1.834862,0.444444,0.358423,0.000000,0.352113,0.350877,1.444043,...,1.785714,1.219512,4.6875,7.407407,36.363636,23.782235,15.691489,0.909091,3.389831,0.381679
1,4.854369,3.809524,0.483092,2.752294,0.888889,1.075269,1.127820,0.704225,1.403509,1.444043,...,0.892857,1.219512,7.8125,7.407407,31.818182,55.587393,42.819149,2.727273,0.847458,0.763359
2,3.883495,1.904762,0.000000,1.376147,0.000000,1.075269,1.127820,1.056338,2.105263,1.444043,...,1.785714,0.000000,7.8125,0.000000,4.545455,36.962751,51.329787,1.818182,0.847458,0.381679
3,3.883495,2.380952,0.483092,1.376147,1.777778,1.433692,0.751880,0.352113,1.754386,1.805054,...,1.785714,3.658537,10.9375,7.407407,31.818182,0.286533,0.000000,2.727273,4.237288,61.450382
4,0.970874,1.428571,0.483092,0.458716,0.000000,0.000000,0.000000,0.352113,0.350877,0.000000,...,1.785714,2.439024,1.5625,0.000000,18.181818,0.286533,0.000000,0.000000,0.000000,0.381679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,2.912621,2.380952,0.483092,2.293578,0.444444,0.716846,0.751880,0.352113,2.105263,0.722022,...,2.678571,6.097561,7.8125,3.703704,59.090909,0.573066,0.265957,4.545455,0.847458,1.145038
64,5.825243,3.333333,1.932367,1.834862,0.888889,0.358423,1.503759,1.056338,0.701754,0.722022,...,69.642857,0.000000,3.1250,0.000000,13.636364,0.000000,0.000000,0.000000,1.694915,0.381679
65,1.941748,0.476190,0.000000,0.917431,0.888889,0.716846,0.751880,1.408451,0.350877,1.444043,...,3.571429,71.951220,3.1250,3.703704,31.818182,0.000000,0.265957,0.909091,0.847458,0.763359
66,2.912621,1.428571,0.000000,2.293578,0.888889,1.433692,0.751880,1.056338,1.403509,1.083032,...,6.250000,3.658537,81.2500,3.703704,50.000000,0.000000,0.531915,0.000000,1.694915,0.381679


In [11]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

71.96883003029274


### Dissimilar Videos

In [32]:
df = videoSimilarity("test-videos/test9.mp4", "test-videos/test11.mp4")
df

fps: 30
Total Number of keyframes saved: 16
fps: 25
Total Number of keyframes saved: 32


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.414938,1.941748,0.000000,0.533333,2.205882,1.766784,6.349206,1.880878,2.752294,1.176471,...,0.000000,0.740741,1.086957,3.773585,1.809955,1.554404,2.347418,0.404858,5.194805,2.439024
1,1.659751,2.912621,0.898204,1.866667,4.411765,1.413428,5.555556,1.880878,4.128440,0.235294,...,0.000000,0.740741,0.543478,1.415094,1.357466,3.108808,2.347418,0.404858,3.896104,3.658537
2,0.829876,4.854369,0.299401,0.800000,0.000000,1.413428,4.761905,0.313480,1.834862,1.176471,...,0.408163,0.370370,2.173913,1.886792,2.262443,2.072539,1.408451,0.809717,6.493506,7.317073
3,2.074689,3.883495,0.898204,2.133333,5.882353,2.826855,4.761905,1.253918,0.917431,0.235294,...,0.816327,1.481481,2.717391,1.415094,2.714932,2.590674,5.164319,0.404858,7.792208,8.536585
4,2.074689,0.970874,0.000000,1.600000,3.676471,1.413428,4.761905,1.253918,0.917431,0.941176,...,0.000000,0.370370,0.543478,1.415094,2.714932,1.554404,0.938967,1.214575,2.597403,0.000000
5,1.244813,7.766990,1.197605,0.533333,5.147059,1.413428,3.968254,1.253918,1.376147,0.941176,...,0.000000,0.740741,0.543478,1.886792,3.167421,2.590674,4.225352,1.619433,2.597403,2.439024
6,1.244813,3.883495,0.898204,0.800000,4.411765,0.706714,3.174603,0.313480,1.376147,0.705882,...,0.000000,0.370370,1.630435,1.886792,1.809955,1.554404,3.755869,2.429150,3.896104,3.658537
7,1.244813,6.796117,0.299401,0.533333,4.411765,1.413428,1.587302,0.313480,1.834862,1.411765,...,0.000000,0.370370,1.630435,2.830189,3.167421,1.554404,2.816901,1.214575,2.597403,2.439024
8,2.074689,2.912621,0.299401,1.333333,6.617647,1.060071,3.174603,0.313480,1.376147,0.941176,...,0.816327,0.000000,1.086957,2.830189,2.714932,2.590674,2.816901,2.024291,3.896104,9.756098
9,5.394191,2.912621,0.299401,2.133333,6.617647,2.120141,1.587302,0.000000,1.376147,2.117647,...,0.408163,0.370370,2.173913,2.358491,6.334842,7.772021,2.347418,2.024291,2.597403,6.097561


In [36]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

6.865378122628382


In [7]:
print(df.max(axis=1).sum()/len(df))

6.865378122628382


### -------------------------------------

In [37]:
df = videoSimilarity("test-videos/test4.mp4", "test-videos/test5.mp4")
df

fps: 30
Total Number of keyframes saved: 8
fps: 30
Total Number of keyframes saved: 31


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0.000000,2.409639,0.438596,2.643172,1.507538,0.0,5.294118,0.0,0.0,0.000000,...,0.000000,0.000000,3.378378,0.000000,0.0,0.000000,0.000000,1.388889,1.342282,1.307190
1,0.452489,0.000000,0.000000,0.000000,0.000000,0.0,0.588235,0.0,0.0,0.555556,...,0.370370,0.000000,0.000000,0.000000,0.0,0.719424,0.793651,0.694444,0.671141,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.653595
3,0.000000,0.000000,0.000000,0.440529,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.694444,0.671141,0.000000
4,0.000000,1.606426,0.000000,1.762115,0.000000,0.0,1.176471,0.0,0.0,1.666667,...,1.111111,0.000000,1.351351,1.886792,0.0,0.000000,0.793651,1.388889,3.355705,1.307190
5,0.000000,0.803213,0.438596,1.321586,0.000000,0.0,2.941176,0.0,0.0,0.555556,...,0.000000,0.000000,2.027027,0.000000,0.0,0.000000,0.793651,2.083333,1.342282,1.307190
6,0.452489,0.803213,0.000000,0.881057,1.005025,0.0,1.176471,0.0,0.0,0.000000,...,0.000000,0.000000,1.351351,0.943396,0.0,0.000000,1.587302,2.083333,1.342282,0.653595
7,0.000000,0.000000,0.000000,0.000000,0.502513,0.0,0.000000,0.0,0.0,1.111111,...,0.000000,0.884956,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [38]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

2.166622178157413


In [18]:
print(df.max(axis=1).sum()/len(df))

2.166622178157413


### Same Videos

In [7]:
df = videoSimilarity("test-videos/test9.mp4", "test-videos/test9.mp4")
df

fps: 30
Total Number of keyframes saved: 16
fps: 30
Total Number of keyframes saved: 16


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,100.000000,4.705882,16.253444,3.313253,2.512563,2.754821,2.616279,1.960784,1.315789,1.546392,1.648352,1.030928,1.648352,0.0,2.760736,3.246753
1,3.519062,100.000000,4.683196,7.530120,9.547739,0.550964,0.000000,0.784314,0.877193,2.577320,1.648352,0.687285,0.000000,0.0,0.306748,1.298701
2,18.181818,5.098039,100.000000,3.915663,1.005025,2.203857,1.744186,1.176471,0.877193,2.577320,2.197802,0.343643,1.648352,0.0,3.067485,1.948052
3,3.225806,9.803922,4.407713,100.000000,1.507538,1.652893,0.872093,3.137255,2.192982,3.608247,2.747253,1.718213,2.197802,0.0,0.306748,1.298701
4,2.052786,5.882353,0.826446,0.903614,100.000000,1.377410,2.034884,1.960784,2.192982,2.061856,1.648352,1.030928,2.197802,0.0,0.613497,0.649351
5,2.639296,2.352941,1.101928,0.903614,3.015075,100.000000,65.988372,54.509804,43.421053,29.896907,19.230769,4.467354,7.142857,0.0,0.613497,1.948052
6,2.639296,1.960784,1.652893,1.506024,2.010050,62.809917,100.000000,67.843137,51.754386,34.020619,21.978022,3.780069,6.043956,0.0,0.920245,1.948052
7,2.639296,2.352941,1.652893,1.204819,1.507538,37.190083,50.000000,100.000000,65.789474,39.690722,25.274725,5.154639,9.340659,0.0,0.920245,1.298701
8,1.466276,1.568627,1.101928,0.602410,1.507538,26.997245,34.011628,58.823529,100.000000,48.453608,32.417582,6.872852,9.340659,0.0,1.226994,2.597403
9,1.466276,1.176471,3.030303,3.915663,1.507538,14.049587,18.023256,29.019608,42.105263,100.000000,52.197802,8.591065,13.186813,0.0,1.533742,1.948052


In [8]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

100.0
